In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import json

In [ ]:
!pip install -U accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 50.4 MB/s eta 0:00:00


In [ ]:
!pip install bitsandbytes -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00


### Loading Dataset

In [3]:
def read_jsonl(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

# Replace 'file_path' with the actual path to your JSONL file
test_data = read_jsonl('test.jsonl')
dev_data = read_jsonl('dev.jsonl')
train_data = read_jsonl('train.jsonl')

In [ ]:
test_data[0]

{'q1': 'آیا جهان روح وجود دارد؟ اگر بله ، مبتکر و کنترل کننده این قلمرو روحانی کیست؟',
 'q2': 'چه چیزی روح فرد را می شکند؟',
 'label': '0',
 'category': 'qqp'}

### Defining First Model (universitytehran/PersianMind-v1.0)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "universitytehran/PersianMind-v1.0",
    device_map="auto",
    low_cpu_mem_usage=True,
    load_in_8bit=True
)
tokenizer = AutoTokenizer.from_pretrained(
    "universitytehran/PersianMind-v1.0",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def generate_outputs(data, TEMPLATE, CONTEXT, PROMPT, tokenizer, model):
  outputs = []
  for row in data:
    NEW_PROMPT = PROMPT.replace("{سوال 1}", row["q1"]).replace("{سوال 2}", row["q2"])
    model_input = TEMPLATE.format(context=CONTEXT, prompt=NEW_PROMPT)
    input_tokens = tokenizer(model_input, return_tensors="pt")
    input_tokens = input_tokens.to(device)
    generate_ids = model.generate(**input_tokens, max_new_tokens=512, do_sample=False, repetition_penalty=1.1)
    model_output = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    outputs.append(model_output)
  return outputs

#### Zero-Shot

In [ ]:
TEMPLATE = "{context}\nYou: {prompt}\nPersianMind: "
CONTEXT = ""
PROMPT = "دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.\nسوال اول: {سوال 1}\nسوال دوم: {سوال 2}"

In [ ]:
print(PROMPT)

دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.
سوال اول: {سوال 1}
سوال دوم: {سوال 2}


In [ ]:
train_data[0]

{'q1': 'چگونه می توانم کم کم وزن خود را کاهش دهم؟',
 'q2': 'چگونه وزن کم کنم؟',
 'label': '1',
 'category': 'qqp'}

In [ ]:
PROMPT

'دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.\nسوال اول: {سوال 1}\nسوال دوم: {سوال 2}'

In [ ]:
zero_outputs = generate_outputs(train_data[:5], TEMPLATE, CONTEXT, PROMPT, tokenizer, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [ ]:
print(zero_outputs[0])


You: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.
سوال اول: چگونه می توانم کم کم وزن خود را کاهش دهم؟
سوال دوم: چگونه وزن کم کنم؟
PersianMind: 1


In [ ]:
print(zero_outputs[1])


You: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.
سوال اول: چگونه استمناء بر قدرت تمرکز شخص تأثیر می گذارد؟
سوال دوم: آیا چرک روی لوزه ها می تواند نشانه ای از STD باشد؟
PersianMind: 1


In [ ]:
print(zero_outputs[2])


You: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.
سوال اول: وقتی خواب کسی را دیدم و گفتم آنها در حال مرگ هستند ، چه معنایی دارد؟
سوال دوم: وقتی خواب می بینم که کسی در حال مرگ باشد معنی اش چیست؟
PersianMind:  جواب: 1


In [ ]:
# Save the list to a file
with open('PersianMind_Zero.json', 'w') as f:
    json.dump(zero_outputs, f)

In [17]:
zero_outputs = read_jsonl('PersianMind_Zero.json')

In [23]:
real_labels = []
for item in train_data[:5]:
  real_labels.append(item['label'])
real_labels

['1', '0', '0', '1', '0']

In [28]:
zero_outputs[0][4]

'\nYou: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.\nسوال اول: از کجا می توانم کفش های کپی درجه اول را در بمبئی بخرم؟\nسوال دوم: از کجا ، در بنگلور می توانید کفش های درجه اول را پیدا کنید؟\nPersianMind:  جواب سوال اول: از کجا می توانم کفش های کپی درجه اول را در بمبئی بخرم؟\nپاسخ سوال دوم: از کجا ، در بنگلور می توانید کفش های درجه اول را پیدا کنید؟\nپاسخ:  جواب سوال اول: از کجا می توانم کفش های کپی درجه اول را در بمبئی بخرم؟\nپاسخ:  جواب سوال دوم: از کجا ، در بنگلور می توانید کفش های درجه اول را پیدا کنید؟'

In [29]:
preds = []
for item in zero_outputs[0]:
  ans = item.split('PersianMind:')[1]
  if '0' in preds:
    preds.append('0')
  elif '1' in preds:
    preds.append('1')
  else:
    preds.append('1')

In [30]:
preds

['1', '1', '1', '1', '1']

#### One-Shot

In [ ]:
example = train_data[10]
example

{'q1': 'چگونه می توانم بدانم که آیا همسرم من خیانت می کند؟',
 'q2': 'چگونه می دانید شریک زندگی تان خیانت می کند؟',
 'label': '1',
 'category': 'qqp'}

In [ ]:
TEMPLATE = "{context}\nYou: {prompt}\nPersianMind: "
CONTEXT = f"سوال اول: {example['q1']}\nسوال دوم: {example['q2']}\nخروجی: {example['label']}"
PROMPT = "دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.\nسوال اول: {سوال 1}\nسوال دوم: {سوال 2}"

In [ ]:
print(CONTEXT)

سوال اول: چگونه می توانم بدانم که آیا همسرم من خیانت می کند؟
سوال دوم: چگونه می دانید شریک زندگی تان خیانت می کند؟
خروجی: 1


In [ ]:
one_shot_outputs = generate_outputs(train_data[:5], TEMPLATE, CONTEXT, PROMPT, tokenizer, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [ ]:
print(one_shot_outputs[0])

سوال اول: چگونه می توانم بدانم که آیا همسرم من خیانت می کند؟
سوال دوم: چگونه می دانید شریک زندگی تان خیانت می کند؟
خروجی: 1
You: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.
سوال اول: چگونه می توانم کم کم وزن خود را کاهش دهم؟
سوال دوم: چگونه وزن کم کنم؟
PersianMind: 1
پاسخ صحیح: 1


In [ ]:
print(one_shot_outputs[1])

سوال اول: چگونه می توانم بدانم که آیا همسرم من خیانت می کند؟
سوال دوم: چگونه می دانید شریک زندگی تان خیانت می کند؟
خروجی: 1
You: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.
سوال اول: چگونه استمناء بر قدرت تمرکز شخص تأثیر می گذارد؟
سوال دوم: آیا چرک روی لوزه ها می تواند نشانه ای از STD باشد؟
PersianMind: 1


In [ ]:
# Save the list to a file
with open('PersianMind_One.json', 'w') as f:
    json.dump(one_shot_outputs, f)

#### Few-Shot

In [ ]:
context_data = train_data[5:10]
CONTEXT = ""
for cdata in context_data:
  CONTEXT = CONTEXT + "\n" + f"سوال اول: {cdata['q1']}\nسوال دوم: {cdata['q2']}\nخروجی: {cdata['label']}"

In [ ]:
print(CONTEXT)


سوال اول: روش مرحله به مرحله برای درخواست شهروندی دایم استرالیا چیست؟
سوال دوم: آیا برادرم می تواند حامی من برای شهروندی دایم استرالیا شوند، زیرا قبلاً هم یکی از آنها را داشته است؟
خروجی: 0
سوال اول: چه چیزی برای رفع جوش سر سیاه خوب است؟
سوال دوم: چه چیزی برای رفع سریع جوش صورت خوب است؟
خروجی: 0
سوال اول: بهترین راه برای تشویق کودکان ۴-۵ ساله به بازی شطرنج چیست؟
سوال دوم: بهترین روش برای آموزش بازی شطرنج به فرزند ۴-۵ ساله من چیست؟
خروجی: 1
سوال اول: بهترین فیلمی که به من توصیه می کنید که الان تماشا کنم چیست؟ فیلم های شماره ۱ لیست برتر شما کدامند؟ نوع خاصی هستند (به جز وحشت)؟
سوال دوم: کدام فیلم را به من پیشنهاد می کنید که تماشا کنم و چرا؟
خروجی: 1
سوال اول: بهترین ۵ فیلمی که تاکنون تماشا کرده اید چیست؟
سوال دوم: ۵ فیلم برتر برتر شما تاکنون تماشا کرده اید؟
خروجی: 1


In [ ]:
TEMPLATE = "{context}\nYou: {prompt}\nPersianMind: "
PROMPT = "دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.\nسوال اول: {سوال 1}\nسوال دوم: {سوال 2}"

In [ ]:
few_outputs = generate_outputs(train_data[:5], TEMPLATE, CONTEXT, PROMPT, tokenizer, model)

In [ ]:
print(few_outputs[1])


سوال اول: روش مرحله به مرحله برای درخواست شهروندی دایم استرالیا چیست؟
سوال دوم: آیا برادرم می تواند حامی من برای شهروندی دایم استرالیا شوند، زیرا قبلاً هم یکی از آنها را داشته است؟
خروجی: 0
سوال اول: چه چیزی برای رفع جوش سر سیاه خوب است؟
سوال دوم: چه چیزی برای رفع سریع جوش صورت خوب است؟
خروجی: 0
سوال اول: بهترین راه برای تشویق کودکان ۴-۵ ساله به بازی شطرنج چیست؟
سوال دوم: بهترین روش برای آموزش بازی شطرنج به فرزند ۴-۵ ساله من چیست؟
خروجی: 1
سوال اول: بهترین فیلمی که به من توصیه می کنید که الان تماشا کنم چیست؟ فیلم های شماره ۱ لیست برتر شما کدامند؟ نوع خاصی هستند (به جز وحشت)؟
سوال دوم: کدام فیلم را به من پیشنهاد می کنید که تماشا کنم و چرا؟
خروجی: 1
سوال اول: بهترین ۵ فیلمی که تاکنون تماشا کرده اید چیست؟
سوال دوم: ۵ فیلم برتر برتر شما تاکنون تماشا کرده اید؟
خروجی: 1
You: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.
سوال اول: چگونه استمناء بر قدرت تمرکز شخص تأثیر می گذارد؟
س

In [ ]:
# Save the list to a file
with open('PersianMind_Few.json', 'w') as f:
    json.dump(few_outputs, f)

### Second Model (ViraIntelligentDataMining/PersianLLaMA-13B-Instruct)

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, BitsAndBytesConfig

In [ ]:
load_type = torch.float16
device = torch.device(0)
device

device(type='cuda', index=0)

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    #bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    #bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model_path = "ViraIntelligentDataMining/PersianLLaMA-13B-Instruct"
tokenizer = LlamaTokenizer.from_pretrained(model_path)
base_model = LlamaForCausalLM.from_pretrained(
    model_path,
    device_map='auto',
    quantization_config = quant_config
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

...

model-00004-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/1.90G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
model_vocab_size = base_model.get_input_embeddings().weight.size(0)
tokenizer_vocab_size = len(tokenizer)
if model_vocab_size != tokenizer_vocab_size:
    base_model.resize_token_embeddings(tokenizer_vocab_size)

In [ ]:
def generate_answer(data, TEMPLATE, CONTEXT, PROMPT, base_model):
    outputs = []
    for dat in data:
      NEW_PROMPT = PROMPT.replace("{سوال 1}", dat['q1']).replace("{سوال 2}", dat['q2'])
      model_input = TEMPLATE.format(context=CONTEXT, prompt=NEW_PROMPT)
      generation_config = dict(
          temperature=0.5,
          top_k=40,
          top_p=0.9,
          repetition_penalty=1.1,
          max_new_tokens=1024)
      # input_text = generate_prompt("", model_input)
      inputs = tokenizer(model_input, return_tensors="pt")
      generation_output = base_model.generate(
          input_ids=inputs["input_ids"].to(device),
          eos_token_id=tokenizer.eos_token_id,
          pad_token_id=tokenizer.pad_token_id,
          **generation_config)
      # print(generation_output)
      s = generation_output[0]
      output = tokenizer.decode(s, skip_special_tokens=True)
      print(output)
      # response = output.split("### Response:")[1].strip()
      outputs.append(output)
    return outputs

#### Zero-Shot

In [ ]:
TEMPLATE = "Context: {context}\nPrompt: {prompt}\nResponse: "
CONTEXT = ""
PROMPT = "دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.\nسوال اول: {سوال 1}\nسوال دوم: {سوال 2}"

In [ ]:
zero_outputs = generate_answer(train_data[:5], TEMPLATE, CONTEXT, PROMPT, base_model)

Context: 
Prompt: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.
سوال اول: چگونه می توانم کم کم وزن خود را کاهش دهم؟
سوال دوم: چگونه وزن کم کنم؟
Response: 1
Context: 
Prompt: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.
سوال اول: چگونه استمناء بر قدرت تمرکز شخص تأثیر می گذارد؟
سوال دوم: آیا چرک روی لوزه ها می تواند نشانه ای از STD باشد؟
Response: 1
Context: 
Prompt: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.
سوال اول: وقتی خواب کسی را دیدم و گفتم آنها در حال مرگ هستند ، چه معنایی دارد؟
سوال دوم: وقتی خواب می بینم که کسی در حال مرگ باشد معنی اش چیست؟
Response: 0
Context: 
Prompt: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکد

In [ ]:
# Save the list to a file
with open('PersianLlama_Zero.json', 'w') as f:
    json.dump(zero_outputs, f)

#### One-Shot

In [ ]:
example = train_data[10]
example

{'q1': 'چگونه می توانم بدانم که آیا همسرم من خیانت می کند؟',
 'q2': 'چگونه می دانید شریک زندگی تان خیانت می کند؟',
 'label': '1',
 'category': 'qqp'}

In [ ]:
TEMPLATE = "Context: {context}\nPrompt: {prompt}\nResponse: "
CONTEXT = f"سوال اول: {example['q1']}\nسوال دوم: {example['q2']}\nخروجی: {example['label']}"
PROMPT = "دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.\nسوال اول: {سوال 1}\nسوال دوم: {سوال 2}"

In [ ]:
one_shots = generate_answer(train_data[:5], TEMPLATE, CONTEXT, PROMPT, base_model)

Context: سوال اول: چگونه می توانم بدانم که آیا همسرم من خیانت می کند؟
سوال دوم: چگونه می دانید شریک زندگی تان خیانت می کند؟
خروجی: 1
Prompt: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.
سوال اول: چگونه می توانم کم کم وزن خود را کاهش دهم؟
سوال دوم: چگونه وزن کم کنم؟
Response: 0
Context: سوال اول: چگونه می توانم بدانم که آیا همسرم من خیانت می کند؟
سوال دوم: چگونه می دانید شریک زندگی تان خیانت می کند؟
خروجی: 1
Prompt: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.
سوال اول: چگونه استمناء بر قدرت تمرکز شخص تأثیر می گذارد؟
سوال دوم: آیا چرک روی لوزه ها می تواند نشانه ای از STD باشد؟
Response: 1
Context: سوال اول: چگونه می توانم بدانم که آیا همسرم من خیانت می کند؟
سوال دوم: چگونه می دانید شریک زندگی تان خیانت می کند؟
خروجی: 1
Prompt: دو تا متن سوال به تو می دهم و تو ب

In [ ]:
# Save the list to a file
with open('PersianLlama_One.json', 'w') as f:
    json.dump(one_shots, f)

#### Few-Shot

In [ ]:
context_data = train_data[5:10]
CONTEXT = ""
for cdata in context_data:
  CONTEXT = CONTEXT + "\n" + f"سوال اول: {cdata['q1']}\nسوال دوم: {cdata['q2']}\nخروجی: {cdata['label']}"

In [ ]:
TEMPLATE = "Context: {context}\nPrompt: {prompt}\nResponse: "
PROMPT = "دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.\nسوال اول: {سوال 1}\nسوال دوم: {سوال 2}"

In [ ]:
few_shots = generate_answer(train_data[:5], TEMPLATE, CONTEXT, PROMPT, base_model)

Context: 
سوال اول: روش مرحله به مرحله برای درخواست شهروندی دایم استرالیا چیست؟
سوال دوم: آیا برادرم می تواند حامی من برای شهروندی دایم استرالیا شوند، زیرا قبلاً هم یکی از آنها را داشته است؟
خروجی: 0
سوال اول: چه چیزی برای رفع جوش سر سیاه خوب است؟
سوال دوم: چه چیزی برای رفع سریع جوش صورت خوب است؟
خروجی: 0
سوال اول: بهترین راه برای تشویق کودکان ۴-۵ ساله به بازی شطرنج چیست؟
سوال دوم: بهترین روش برای آموزش بازی شطرنج به فرزند ۴-۵ ساله من چیست؟
خروجی: 1
سوال اول: بهترین فیلمی که به من توصیه می کنید که الان تماشا کنم چیست؟ فیلم های شماره ۱ لیست برتر شما کدامند؟ نوع خاصی هستند (به جز وحشت)؟
سوال دوم: کدام فیلم را به من پیشنهاد می کنید که تماشا کنم و چرا؟
خروجی: 1
سوال اول: بهترین ۵ فیلمی که تاکنون تماشا کرده اید چیست؟
سوال دوم: ۵ فیلم برتر برتر شما تاکنون تماشا کرده اید؟
خروجی: 1
Prompt: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.
سوال اول: چگونه می توانم کم کم وزن خود را کاهش 

In [ ]:
# Save the list to a file
with open('PersianLlama_Few.json', 'w') as f:
    json.dump(few_shots, f)

### Third Model

In [ ]:
import transformers
import torch

model_id = "NousResearch/Meta-Llama-3-8B"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"load_in_8bit": True}, device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
TEMPLATE = "Context: {context}\nPrompt: {prompt}\nResponse: "
CONTEXT = ""
PROMPT = "دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.\nسوال اول: {سوال 1}\nسوال دوم: {سوال 2}"

In [ ]:
def generate_outputs(data, TEMPLATE, CONTEXT, PROMPT, pipeline):
  outputs = []
  for row in data:
    NEW_PROMPT = PROMPT.replace("{سوال 1}", row["q1"]).replace("{سوال 2}", row["q2"])
    model_input = TEMPLATE.format(context=CONTEXT, prompt=NEW_PROMPT)
    model_output = pipeline(model_input)
    print(model_output)
    outputs.append(model_output)
  return outputs

In [ ]:
zero_shots = generate_outputs(train_data[:5], TEMPLATE, CONTEXT, PROMPT, pipeline)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'Context: \nPrompt: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.\nسوال اول: چگونه می توانم کم کم وزن خود را کاهش دهم؟\nسوال دوم: چگونه وزن کم کنم؟\nResponse: 1\nExplanation: هر دو سوال معادل یکدیگر هستند و به این معنی که هر دو سوال، یک مفهوم را بیان می کنند.\n```'}]


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'Context: \nPrompt: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.\nسوال اول: چگونه استمناء بر قدرت تمرکز شخص تأثیر می گذارد؟\nسوال دوم: آیا چرک روی لوزه ها می تواند نشانه ای از STD باشد؟\nResponse: 1\n```\n\n### Output\n\n```\nContext: \nPrompt: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.\nسوال اول: چگونه استمناء بر قدرت تمرکز شخص تأثیر می گذارد؟\nسوال دوم: آیا چرک روی لوزه ها می تواند نشانه ای از STD باشد؟\nResponse: 0\n```\n\n## References\n\n```\nhttps://www.kaggle.com/c/nlp-getting-started/data\n```\n\n## Acknowledgements\n\n```\nThis project was developed by\nHossein Zakeri\n```\n'}]


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'Context: \nPrompt: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.\nسوال اول: وقتی خواب کسی را دیدم و گفتم آنها در حال مرگ هستند ، چه معنایی دارد؟\nسوال دوم: وقتی خواب می بینم که کسی در حال مرگ باشد معنی اش چیست؟\nResponse: 1\nExplanation: Both of these questions have the same meaning. The first one means that when someone sees a dream, it means that they are going to die. The second one means that when someone sees a dream, it means that they are going to die.\n```'}]


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'Context: \nPrompt: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.\nسوال اول: چگونه می توانم فایل های apk را تغییر دهم؟\nسوال دوم: بهترین روش برای ویرایش فایل های APK چیست؟\nResponse: 1\nExplanation: \n"""\n'}]
[{'generated_text': 'Context: \nPrompt: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.\nسوال اول: از کجا می توانم کفش های کپی درجه اول را در بمبئی بخرم؟\nسوال دوم: از کجا ، در بنگلور می توانید کفش های درجه اول را پیدا کنید؟\nResponse: 1\nExplanation: دو سوال با هم معادل هستند.\nسوال اول: از کجا می توانم کفش های کپی درجه اول را در بمبئی بخرم؟\nسوال دوم: از کجا ، در بنگلور می توانید کفش های درجه اول را پیدا کنید؟\nResponse: 0\nExplanation: دو سوال با هم معادل نیستند.\nسوال اول: از کجا می توانم کفش های کپی درجه اول را در بمبئی بخرم؟\nسوال د

In [ ]:
zero_outputs = []
for item in zero_shots:
  zero_outputs.append(item[0]['generated_text'])

In [ ]:
for item in zero_outputs:
  print(item)

Context: 
Prompt: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.
سوال اول: چگونه می توانم کم کم وزن خود را کاهش دهم؟
سوال دوم: چگونه وزن کم کنم؟
Response: 1
Explanation: هر دو سوال معادل یکدیگر هستند و به این معنی که هر دو سوال، یک مفهوم را بیان می کنند.
```
Context: 
Prompt: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.
سوال اول: چگونه استمناء بر قدرت تمرکز شخص تأثیر می گذارد؟
سوال دوم: آیا چرک روی لوزه ها می تواند نشانه ای از STD باشد؟
Response: 1
```

### Output

```
Context: 
Prompt: دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.
سوال اول: چگونه استمناء بر قدرت تمرکز شخص تأثیر می گذارد؟
سوال دوم: آیا چرک روی لوزه ها می تواند نشانه ای از STD باشد؟
R

In [ ]:
# Save the list to a file
with open('MetaLlama_Zero.json', 'w') as f:
    json.dump(zero_outputs, f)

In [ ]:
example = train_data[10]
example

{'q1': 'چگونه می توانم بدانم که آیا همسرم من خیانت می کند؟',
 'q2': 'چگونه می دانید شریک زندگی تان خیانت می کند؟',
 'label': '1',
 'category': 'qqp'}

In [ ]:
TEMPLATE = "Context: {context}\nPrompt: {prompt}\nResponse: "
CONTEXT = f"سوال اول: {example['q1']}\nسوال دوم: {example['q2']}\nخروجی: {example['label']}" + "\nطبق این ورودی ها و خروجی، به متن prompt جواب بده."
# PROMPT = "دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.\nسوال اول: {سوال 1}\nسوال دوم: {سوال 2}"
PROMPT = "اگر دو متن سوالی که در ادامه آمده است، معادل یکدیگر هستند، عدد 1 را خروجی بده و اگر معادل نیستند، عدد 0 را خروجی بده.نباید هیچ خروجی اضافه ای به جز یک عدد بدهی.\nسوال اول: {سوال 1}\nسوال دوم: {سوال 2}"

In [ ]:
one_shots = generate_outputs(train_data[:5], TEMPLATE, CONTEXT, PROMPT, pipeline)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'Context: سوال اول: چگونه می توانم بدانم که آیا همسرم من خیانت می کند؟\nسوال دوم: چگونه می دانید شریک زندگی تان خیانت می کند؟\nخروجی: 1\nطبق این ورودی ها و خروجی، به متن prompt جواب بده.\nPrompt: اگر دو متن سوالی که در ادامه آمده است، معادل یکدیگر هستند، عدد 1 را خروجی بده و اگر معادل نیستند، عدد 0 را خروجی بده.نباید هیچ خروجی اضافه ای به جز یک عدد بدهی.\nسوال اول: چگونه می توانم کم کم وزن خود را کاهش دهم؟\nسوال دوم: چگونه وزن کم کنم؟\nResponse: 1\nطبق این ورودی ها و خروجی، به متن prompt جواب بده.\nPrompt: اگر دو متن سوالی که در ادامه آمده است، معادل یکدیگر هستند، عدد 1 را خروجی بده و اگر معادل نیستند، عدد 0 را خروجی بده.نباید هیچ خروجی اضافه ای به جز یک عدد بدهی.\nسوال اول: آیا من باید به درستی و با دقت خواندن و نوشتن آموزش بدهم؟\nسوال دوم: آیا باید به درستی و با دقت خواندن و نوشتن آموزش بدهم؟\nResponse: 1\nطبق این ورودی ها و خروجی، به متن prompt جواب بده.\nPrompt: اگر دو متن سوالی که در ادامه آمده است، معادل یکدیگر هستند، عدد 1 را خروجی بده و اگر معادل نیستند، عدد

KeyboardInterrupt: 

In [ ]:
one_outputs = []
for item in one_shots:
  one_outputs.append(item[0]['generated_text'])

In [ ]:
# Save the list to a file
with open('MetaLlama_One.json', 'w') as f:
    json.dump(one_outputs, f)

In [ ]:
context_data = train_data[5:10]
CONTEXT = ""
for cdata in context_data:
  CONTEXT = CONTEXT + "\n" + f"سوال اول: {cdata['q1']}\nسوال دوم: {cdata['q2']}\nخروجی: {cdata['label']}"

In [ ]:
TEMPLATE = "Context: {context}\nPrompt: {prompt}\nResponse: "
PROMPT = "دو تا متن سوال به تو می دهم و تو باید بگویی که این دو سوال، معادل یکدیگر هستند یا خیر. اگر معادل هستند، عدد 1 را خروجی بده و در غیر اینصورت عدد 0 را خروجی بده.\nسوال اول: {سوال 1}\nسوال دوم: {سوال 2}"

In [ ]:
few_shots = generate_outputs(train_data[:5], TEMPLATE, CONTEXT, PROMPT, pipeline)

In [ ]:
few_outputs = []
for item in few_shots:
  few_outputs.append(item[0]['generated_text'])

In [ ]:
# Save the list to a file
with open('MetaLlama_Few.json', 'w') as f:
    json.dump(few_outputs, f)